In [2]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import random as rand
from tqdm import tqdm
import random as rand

# Visualizaciones
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree

import sys
import os

# Agregar la carpeta 'src' al sys.path
src_path = os.path.abspath("../src")
if src_path not in sys.path:
    sys.path.append(src_path)
    import support_LinearRegression as sp



# Para realizar la regresión lineal y la evaluación del modelo
# -----------------------------------------------------------------------
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import KFold,LeaveOneOut, cross_val_score

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("../data/df_te.csv")
df.sample(5)

,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,fuelType,brand,notRepairedDamage,powerCV_robust,kilometer_robust
164597,5722.744529,5722.69042,950.0,5723.869135,2912.480757,1769.244434,4654.329823,4702.257069,3226.667431,3346.962677,-0.782609,-5.8
318132,5722.744529,5722.69042,4830.0,5721.515220,5212.719911,5236.859483,4654.329823,4702.257069,4123.442393,6707.871077,-0.130435,0.0
196940,5722.744529,5722.69042,4750.0,5723.869135,9342.620539,1769.244434,4654.329823,4702.257069,8769.608410,6707.871077,0.507246,0.0
296418,5722.744529,5722.69042,1800.0,5723.869135,5212.719911,1769.244434,4654.329823,4702.257069,5407.374502,6707.871077,-0.188406,0.0
47392,5722.744529,5722.69042,1780.0,5721.515220,2912.480757,5236.859483,4654.329823,4702.257069,3073.553291,6707.871077,-0.782609,-2.4


In [4]:
x=df.drop("price",axis=1)
y=df[["price"]]

In [6]:
x_train, x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=42)

In [ ]:
# Parámetros a evaluar
params_arbol = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [10, 50, 100, 200],
    'min_samples_leaf': [10, 50, 100, 200],
    'max_leaf_nodes': [4, 6, 8, 10, 20, 30]
}
grid_search_arbol=GridSearchCV(DecisionTreeRegressor(),
                               param_grid=params_arbol,
                               cv=5,
                               scoring="neg_mean_squared_error",
                               n_jobs=-1)

grid_search_arbol.fit(x_train,y_train)

In [ ]:
modelo_arbol_final=grid_search_arbol.best_estimator_

In [ ]:
sp.graficar_arbol_decision(modelo_arbol_final,x.columns)

In [ ]:
df_resultados_arbol=pd.DataFrame(grid_search_arbol.cv_results_).sort_values(by="rank_test_score",ascending=True)
df_resultados_arbol_filtrado=df_resultados_arbol.filter(like="param").drop(columns="params")
df_resultados_arbol_filtrado["MSE"]=df_resultados_arbol["mean_test_score"]
df_resultados_arbol_filtrado